In [4]:
# 4. Automatic Tracing via Environment (Groq Equivalent of wrap_openai)

# We demonstrate wrapping by using a simple Groq call without any traceable decorators, 
# relying purely on the environment variables set outside this notebook.

def simple_groq_call(user_input: str):
    messages = [
        {"role": "system", "content": "You are a concise summarizer."},
        {"role": "user", "content": user_input}
    ]
    # This LLM call is automatically traced because LANGSMITH_TRACING=true is set in .env
    response = llm_client.invoke(messages) 
    return response.content

print("\n--- Automatic Tracing Example ---\n")
question_auto = "Summarize the key difference between @traceable and the with trace() context manager."
ai_answer_auto = simple_groq_call(question_auto)
print(ai_answer_auto)


--- Automatic Tracing Example ---

`@traceable` is not a standard Python decorator, but assuming it's similar to other tracing decorators, the key difference between `@traceable` and the `with trace()` context manager is:

* `@traceable` is a function decorator that enables tracing for a specific function.
* `with trace()` is a context manager that enables tracing for a block of code within the `with` statement.

In other words, `@traceable` is function-scoped, while `with trace()` is block-scoped.


In [3]:
# 3. Execution of Context Manager Example

question_cm = "How do I trace complex logic using the LangSmith Context Manager?"
ai_answer_cm = langsmith_rag(question_cm)

print("--- Question (Context Manager) ---\n")
print(question_cm)
print("\n--- Answer ---\n")
print(ai_answer_cm)

--- Question (Context Manager) ---

How do I trace complex logic using the LangSmith Context Manager?

--- Answer ---

I don't know how to trace complex logic using the LangSmith Context Manager, as the provided context only discusses using the @traceable decorator for tracing. The context does not mention the LangSmith Context Manager or its usage. It may be a separate feature or functionality that is not covered in the given information.


In [2]:
# 2. RAG Functions: Implementing the `with trace()` Context Manager Tweak

@traceable(run_type="retriever", name="Retrieve Documents")
def retrieve_documents(question: str):
    return retriever.invoke(question)   

# TWEAK: Removed @traceable and replaced it with a manual trace context
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    # START OF TWEAK: Manual trace context manager
    with trace(
        name="Generate Response (Manual Context)",
        run_type="chain", 
        inputs={"question": question, "documents": documents},
        tags=["manual-context-manager", "lesson-3-tweak"] # Custom tags for identification
    ) as ls_trace:
        messages = [
            {"role": "system", "content": RAG_SYSTEM_PROMPT},
            {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
        ]
        
        # Call the LLM (this will be traced automatically by LangChain/Groq)
        response = llm_client.invoke(messages)
        
        # END OF TWEAK: Manually end the trace and write outputs
        ls_trace.end(outputs={"output": response.content}) 
        
        return response


@traceable(run_type="chain", name="Root RAG Pipeline")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content

In [1]:
# 1. Setup Environment and Imports (Groq Ready)
import os
from dotenv import load_dotenv
from langsmith import traceable, trace # trace is the context manager
from langsmith.wrappers import wrap_openai # Needed for Groq/OpenAI client wrapping
from langchain_groq import ChatGroq 
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever, RAG_SYSTEM_PROMPT

# --- Configuration ---
load_dotenv()
MODEL_NAME = "llama-3.3-70b-versatile" # Using the confirmed, working Groq model
# We initialize the Groq client
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

c:\Users\Raghav Gupta\anaconda3\envs\langsmith_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Raghav Gupta\OneDrive\Desktop\Langsmith-MAT496\utils.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
